In [1]:
import numpy as np
import random
import os
import time
import torch 
from torch.autograd import Variable
from copy import deepcopy

In [2]:
from lib.turn_program_into_file import turn_program_into_file
from game.wrappers.game_container import GameContainer

In [3]:
cpus = 4
player_size = 8
game = GameContainer(128, 128)
# Each run taks 2.6 sec per cpu, 38089 = a 24h run.
run_limit = 10
save_cycle = 2
output_dir = "./output/pyramid/"
run = int(time.time())
save_dir = output_dir + str(run) + '/'

In [4]:
# king_hill_ids = list(range(0, cpus))
os.makedirs(os.path.join(output_dir, str(run)))

In [ ]:
# -*- coding: utf-8 -*-
import torch

top = 2

# Use the nn package to define our model and loss function.
p1 = torch.nn.Sequential(
    torch.nn.Linear(top**0, top**1),
    torch.nn.ReLU(),
    torch.nn.Linear(top**1, top**2),
    torch.nn.ReLU(),
    torch.nn.Linear(top**2, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


p2 = torch.nn.Sequential(
    torch.nn.Linear(top**0, top**1),
    torch.nn.ReLU(),
    torch.nn.Linear(top**1, top**2),
    torch.nn.ReLU(),
    torch.nn.Linear(top**2, top**3),
    torch.nn.ReLU(),
    torch.nn.Linear(top**3, top**4),
    torch.nn.ReLU(),
    torch.nn.Linear(top**4, top**5),
    torch.nn.ReLU(),
    torch.nn.Linear(top**5, top**6),
    torch.nn.Sigmoid(),
)


loss_fn = torch.nn.MSELoss()


def return_result(game_result, number):
    result = 0
    
    for i in game_result:
        if i == number:
            result += 1
        
    return result

def create_player(game_board_pt, player_size):
    game_board_np = game_board_pt.detach().numpy()
    
    for i in range(len(game_board_np)):
        game_board_np[i] = 1 if game_board_np[i] > 0.5 else 0
    
    game_board_np = np.array(game_board_np, dtype=np.int)
    
    return game_board_np


def run_game(home, away, rounds, player_size):
    home = np.reshape(home, (player_size, player_size))
    away = np.reshape(away, (player_size, player_size))
    
    game.add_players(home, away)
    game.launch(rounds)
    final_board = game.gol.grid()
    final_board = np.array(final_board).flatten()
    
    return return_result(final_board, 1)

def create_loss_board(home, away, rounds, player_size):
    org_score = run_game(home, away, rounds, player_size)
    loss_board = deepcopy(home)
    
    for i in range(player_size**2):
        tile_board = deepcopy(home)
        tile_board[i] = 0 if home[i] == 1 else 1
        tile_result = run_game(tile_board, away, rounds, player_size)
        if tile_result >= org_score:
            loss_board[i] = tile_board[i]
    
    return torch.tensor(loss_board, dtype=torch.float), org_score

learning_rate = 0.001
optimizer_p1 = torch.optim.Adam(p1.parameters(), lr=learning_rate)
optimizer_p2 = torch.optim.Adam(p2.parameters(), lr=learning_rate)

for t in range(10000):
    
    start_p1 = Variable(torch.tensor(torch.rand(1), requires_grad=True))
    start_p2 = Variable(torch.tensor(torch.rand(1), requires_grad=True))
    
    p1_board = p1(start_p1)
    p2_board = p2(start_p2)

    with torch.no_grad():
        p1_board_np = create_player(p1_board.clone(), player_size)
        p2_board_np = create_player(p2_board.clone(), player_size)

        p1_loss_board, p1_score = create_loss_board(p1_board_np, p2_board_np, 500, player_size)
        p2_loss_board, p2_score = create_loss_board(p2_board_np, p1_board_np, 500, player_size)

    loss_p1 = loss_fn(p1_board, p1_loss_board)
    loss_p1.backward()

    
    optimizer_p1.step()
    
    loss_p2 = loss_fn(p2_board, p2_loss_board)
    loss_p2.backward()
    optimizer_p2.step()
    
    optimizer_p1.zero_grad()
    optimizer_p2.zero_grad()
    
    print(loss_p1, loss_p2)
    print(p1_score, p2_score)
    
    if t % 100 == 0:
        print("Saving players:")
        print(p1_board, p2_board)
        turn_program_into_file(np.reshape(p1_board_np, (8,8)), save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
        turn_program_into_file(np.reshape(p2_board_np, (8,8)), save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")

tensor(0.2677) tensor(0.2487)
7 21
Saving players:
tensor([ 0.5070,  0.5194,  0.4697,  0.5064,  0.5437,  0.4796,  0.4621,
         0.5180,  0.4905,  0.5083,  0.5381,  0.5130,  0.4996,  0.5376,
         0.5340,  0.4621,  0.5000,  0.4789,  0.5109,  0.5566,  0.4859,
         0.5345,  0.4929,  0.4573,  0.4858,  0.4937,  0.4942,  0.5531,
         0.5280,  0.5282,  0.5053,  0.5057,  0.4840,  0.5105,  0.4985,
         0.5113,  0.5265,  0.4860,  0.4613,  0.5371,  0.5352,  0.5505,
         0.4851,  0.4802,  0.4757,  0.4801,  0.5199,  0.5200,  0.5302,
         0.5106,  0.5511,  0.4779,  0.5379,  0.5224,  0.4861,  0.5045,
         0.5314,  0.5280,  0.5294,  0.4514,  0.5014,  0.4633,  0.4939,
         0.4807]) tensor([ 0.5217,  0.5001,  0.5203,  0.5076,  0.5089,  0.5099,  0.4398,
         0.4339,  0.5196,  0.4855,  0.4946,  0.4948,  0.4932,  0.5124,
         0.5063,  0.5053,  0.5153,  0.4513,  0.4667,  0.4201,  0.5112,
         0.4885,  0.4315,  0.5248,  0.5452,  0.5290,  0.5629,  0.4625,
        

tensor(1.00000e-02 *
       9.1131) tensor(1.00000e-02 *
       3.4117)
229 370
tensor(1.00000e-02 *
       8.1992) tensor(1.00000e-02 *
       3.2720)
229 370
tensor(1.00000e-02 *
       9.0317) tensor(1.00000e-02 *
       3.2645)
229 370
tensor(1.00000e-02 *
       8.8830) tensor(1.00000e-02 *
       3.2043)
229 370
tensor(1.00000e-02 *
       8.6559) tensor(1.00000e-02 *
       3.1408)
229 370
tensor(1.00000e-02 *
       8.5499) tensor(1.00000e-02 *
       3.0959)
229 370
tensor(1.00000e-02 *
       8.1782) tensor(1.00000e-02 *
       3.0447)
229 370
tensor(1.00000e-02 *
       8.0169) tensor(1.00000e-02 *
       3.0199)
229 370
tensor(1.00000e-02 *
       8.6236) tensor(1.00000e-02 *
       2.9156)
229 370
tensor(1.00000e-02 *
       7.0373) tensor(1.00000e-02 *
       2.8453)
229 370
tensor(1.00000e-02 *
       6.8548) tensor(1.00000e-02 *
       2.8421)
229 370
tensor(1.00000e-02 *
       6.7291) tensor(1.00000e-02 *
       2.7783)
229 370
tensor(1.00000e-02 *
       7.3785) tens

tensor(1.00000e-02 *
       7.0397) tensor(1.00000e-03 *
       8.6148)
211 435
tensor(1.00000e-02 *
       5.0952) tensor(1.00000e-03 *
       8.6562)
211 435
tensor(1.00000e-02 *
       5.0544) tensor(1.00000e-03 *
       8.6224)
211 435
tensor(1.00000e-02 *
       7.5344) tensor(1.00000e-03 *
       9.4320)
211 435
tensor(1.00000e-02 *
       7.4747) tensor(1.00000e-03 *
       9.3053)
211 435
tensor(1.00000e-02 *
       4.3542) tensor(1.00000e-03 *
       9.2048)
211 435
tensor(1.00000e-02 *
       4.3340) tensor(1.00000e-03 *
       8.4422)
211 435
tensor(1.00000e-02 *
       5.4641) tensor(1.00000e-03 *
       8.3494)
211 435
tensor(1.00000e-02 *
       5.4427) tensor(1.00000e-03 *
       8.3420)
211 435
tensor(1.00000e-02 *
       4.8037) tensor(1.00000e-03 *
       9.1065)
211 435
tensor(1.00000e-02 *
       4.7721) tensor(1.00000e-03 *
       9.1747)
211 435
tensor(1.00000e-02 *
       4.7336) tensor(1.00000e-03 *
       9.1922)
211 435
tensor(1.00000e-02 *
       5.2963) tens

In [ ]:
 #print(loss_p1.detach().numpy(), loss_p2.detach().numpy())
    #     if t % 100:
    #         turn_program_into_file(p1_board, save_dir + "p1-cycle-" + str(t) + ".rle", "RKH", "EW", "")
    #         turn_program_into_file(p2_board, save_dir + "p2-cycle-" + str(t) + ".rle", "RKH", "EW", "")